In [1]:
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
%matplotlib inline
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
df1=pd.read_excel('/home/ayushi/Downloads/RE/R1.xlsx',sheet_name='Sheet1', header=1)

In [3]:
df1 = df1.iloc[:-1] # deleting last row

df1['Calendar Year Month']=pd.to_datetime(df1['Calendar Year Month'],format='%Y%m')
df1['year']=df1['Calendar Year Month'].apply(lambda x: x.year) 
df1['month']=df1['Calendar Year Month'].apply(lambda x: x.month)
df1.drop(['Calendar Year Month'],axis=1,inplace=True)

In [4]:
df1.head()

,Product Line Desc (M),Product Class Desc (M),Sales District Desc (M),ABC indicator,Unnamed: 5,USD,USD.1,USD.2,USD.3,USD.4,USD.5,Unnamed: 12,year,month
0,Arresters,#,North East Region,##,0.00,0.0,0.00,0.0,536.34,0.0,0.0,536.34,2017,1
1,Arresters,Blocks,EuropeMidEastAfrica,B,0.00,0.0,0.00,0.0,23040.00,23040.0,15360.0,61440.00,2017,1
2,Arresters,Blocks,EuropeMidEastAfrica,C,60588.00,60588.0,0.00,0.0,25486.20,46656.0,190256.0,322986.20,2017,1
3,Arresters,Blocks,North East Region,C,0.00,0.0,0.00,0.0,5691.00,0.0,0.0,5691.00,2017,1
4,Arresters,Distribution Arresters,APAC,1,9963.52,0.0,9963.52,0.0,0.00,0.0,0.0,9963.52,2017,1


In [5]:
df2=df1[df1['Product Line Desc (M)'].isin(['Cable Accessories', 'Connectors','Construction', 'Insulators'])]

In [6]:
df2.head()

,Product Line Desc (M),Product Class Desc (M),Sales District Desc (M),ABC indicator,Unnamed: 5,USD,USD.1,USD.2,USD.3,USD.4,USD.5,Unnamed: 12,year,month
181,Cable Accessories,URD Accessories,CANADA UTILITY,A,2791.66,481.94,2309.72,0.0,0.00,6328.42,0.00,9120.08,2017,1
182,Cable Accessories,URD Accessories,CANADA UTILITY,B,5512.37,0.00,5512.37,0.0,0.00,0.00,0.00,5512.37,2017,1
183,Cable Accessories,URD Accessories,CANADA UTILITY,D,0.00,0.00,0.00,0.0,0.00,4015.19,4015.19,8030.38,2017,1
184,Cable Accessories,URD Accessories,Communication - West,A,0.00,0.00,0.00,0.0,0.00,0.00,5757.60,5757.60,2017,1
185,Cable Accessories,URD Accessories,Communication - West,B,0.00,0.00,0.00,0.0,429.84,0.00,0.00,429.84,2017,1


In [7]:
le = preprocessing.LabelEncoder()
df2['Sales District Desc (M) '] = le.fit_transform(df2['Sales District Desc (M) '])
df2['Product Line Desc (M)']=le.fit_transform(df2['Product Line Desc (M)'])
one_hot1 = pd.get_dummies(df2['Product Class Desc (M)'])
one_hot2 = pd.get_dummies(df2['ABC indicator'])
df2 = pd.concat([df2, one_hot1], axis=1)
df2=pd.concat([df2,one_hot2], axis=1)
df2.drop(['Product Class Desc (M)','ABC indicator'],axis=1,inplace=True)
df2.drop(['USD','USD.1','USD.2','USD.3','USD.4','USD.5','Unnamed: 12'],inplace=True,axis=1)

/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ayushi/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df2.head()

,Product Line Desc (M),Sales District Desc (M),Unnamed: 5,year,month,#,Aluminium Loops,Atlas,Civil,Distribution Connectors,...,URD Accessories,URD Arresters,Utility Anchors,##,1,A,B,C,D,P
181,0,1,2791.66,2017,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
182,0,1,5512.37,2017,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
183,0,1,0.00,2017,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
184,0,4,0.00,2017,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
185,0,4,0.00,2017,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [9]:
df2.drop(['#','##'],axis=1,inplace=True)

In [10]:
Y=df2['Unnamed: 5']
df2.drop('Unnamed: 5',axis=1,inplace=True)

In [11]:
def xgboost(a):
    X=a
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    model= XGBRegressor(
     learning_rate =0.1,
     n_estimators=1000,
     max_depth=3,
     min_child_weight=5,
     gamma=0,
     subsample=0.8,
     colsample_bytree=0.8,
     objective= 'reg:linear',
     nthread=4,
     scale_pos_weight=1,
     seed=27)
    model.fit(X_train,y_train)
    predictions=model.predict(X_test)
    r2=r2_score(y_test,predictions)
    return predictions, r2

In [12]:
xb_pred=xgboost(df2)[0]

In [14]:
xb_r2=xgboost(df2)[1]
xb_r2

0.525483930293014

In [15]:
lm=linear_model.LinearRegression()
def linear(x):
    X=x
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    model1=lm.fit(X_train, y_train)
    predictions1=lm.predict(X_test)
    r2=r2_score(y_test,predictions1)
    return predictions1,r2

In [16]:
lin_pred=linear(df2)[0]

In [18]:
lin_r2=linear(df2)[1]
lin_r2

0.12751241357085985

In [25]:
def ensemble(x):
    X=x
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    final_pred=0.8*xb_pred+0.5*lin_pred
    r2=r2_score(y_test,final_pred)
    return final_pred,r2

In [28]:
ensem_pred=ensemble(df2)[0]

In [29]:
ensem_r2=ensemble(df2)[1]
ensem_r2

0.4918311896433122